In [1]:
import numpy as np
import pandas as pd
import pickle
import json

In [2]:
df = pd.read_json('proj3_data1.json')
df = pd.concat([df, (pd.read_json('proj3_data2.json')), (pd.read_json('proj3_data3.json'))])
df.index = range(df.shape[0])
df.shape

(18, 8)

In [3]:
df.head(10)

,make,model,body_type,doors,top_speed,acceleration,fuel_consumption,engine
0,Audi,A3,hatchback,5.0,220,7.2,6.0,1.5 TFSI
1,BMW,3 Series,sedan,4.0,240,6.2,NaN,2.0L TwinPower Turbo
2,Volkswagen,Golf,hatchback,5.0,210,8.0,5.0,1.5 TSI
3,Renault,Clio,hatchback,5.0,190,11.0,6.5,1.5 dCi
4,Fiat,500,hatchback,3.0,160,12.9,NaN,0.9 TwinAir
5,Peugeot,208,hatchback,NaN,190,9.9,5.0,1.2 PureTech
6,Volvo,XC60,SUV,5.0,220,8.0,6.0,B4
7,Seat,Leon,hatchback,5.0,240,8.0,NaN,1.5 TSI
8,Opel,Corsa,hatchback,5.0,180,9.9,5.5,1.2 Turbo
9,Ford,Mustang,coupe,2.0,249,5.8,10.2,2.3L EcoBoost


In [4]:
df.to_json('proj3_ex01_all_data.json', orient='columns')

In [5]:
num = []
for x in df.columns[df.isna().any()].tolist():
    tab = [x, df[x].isna().sum()]
    num.append(tab)

In [6]:
print(num)

[['doors', 2], ['fuel_consumption', 4]]


In [7]:
df2 = pd.DataFrame(num)
df2.head()

,0,1
0,doors,2
1,fuel_consumption,4


In [8]:
df2.to_csv('proj3_ex02_no_nulls.csv', index=False, header=False)

In [9]:
with open('proj3_params.json', 'r') as file:
    param_data = json.load(file)

print(param_data)

{'concat_columns': ['make', 'model', 'engine'], 'id_vars': ['make', 'model'], 'join_column': 'engine', 'int_columns': ['doors', 'displacement', 'horsepower', 'cylinders'], 'grouping_column': 'make', 'pivot_index': 'make', 'pivot_columns': 'fuel_type', 'pivot_values': 'fuel_consumption', 'aggregations': [['displacement', 'min'], ['displacement', 'max'], ['fuel_consumption', 'mean']]}


In [10]:
concats = []
for i in range(df.shape[0]):
    con = ""
    for x in param_data['concat_columns']:
        con = con + df[x][i] + " "
    concats.append(con[:-1])

In [11]:
print(concats)

['Audi A3 1.5 TFSI', 'BMW 3 Series 2.0L TwinPower Turbo', 'Volkswagen Golf 1.5 TSI', 'Renault Clio 1.5 dCi', 'Fiat 500 0.9 TwinAir', 'Peugeot 208 1.2 PureTech', 'Volvo XC60 B4', 'Seat Leon 1.5 TSI', 'Opel Corsa 1.2 Turbo', 'Ford Mustang 2.3L EcoBoost', 'Volkswagen Passat 2.0 TDI', 'Porsche 911 3.0L Twin Turbo V6', 'Audi Q5 45 TFSI quattro', 'Volvo S60 T8 Polestar Engineered', 'Renault Megane 1.5 dCi', 'Peugeot 3008 1.6 THP', 'Fiat Panda 1.2 8v', 'Fiat Punto 1.2 8v']


In [12]:
df.insert(len(df.columns.tolist()),"description", concats)

In [13]:
df.head()
df.to_json('proj3_ex03_descriptions.json')

In [14]:
df_more = pd.read_json('proj3_more_data.json')
df_more.head()

,engine,displacement,horsepower,fuel_type,cylinders,emissions_class
0,1.5 TFSI,1498,150,gasoline,4,Euro 6d
1,2.0L TwinPower Turbo,1998,184,gasoline,4,Euro 6d
2,1.5 TSI,1498,150,gasoline,4,Euro 6d
3,1.5 dCi,1461,115,diesel,4,Euro 6d-TEMP
4,0.9 TwinAir,875,85,gasoline,2,Euro 6d-TEMP


In [15]:
df = df.join(df_more.set_index(param_data['join_column']), on=param_data['join_column'], how='left')

In [16]:
df.head()
df.to_json('proj3_ex04_joined.json', orient='columns')

In [17]:
for i, r in df.iterrows():
    name = r['description']
    name = name.lower()
    name = name.replace(" ", "_")
    filtered_series = r.drop('description')
    conc = "proj3_ex05_" + name + ".json";

    filtered_series.to_json(conc)

In [18]:
for i, r in df.iterrows():
    name = r['description']
    name = name.lower()
    name = name.replace(" ", "_")
    filtered_series = r.drop('description')
    for index, value in filtered_series.items():
        if(index in param_data['int_columns']):
            if(pd.isna(filtered_series[index]) is False):
                filtered_series[index] = int(filtered_series[index])
        print(filtered_series[index])
    conc = "proj3_ex05_int_" + name + ".json";
    filtered_series.to_json(conc)

Audi
A3
hatchback
5
220
7.2
6.0
1.5 TFSI
1498
150
gasoline
4
Euro 6d
BMW
3 Series
sedan
4
240
6.2
nan
2.0L TwinPower Turbo
1998
184
gasoline
4
Euro 6d
Volkswagen
Golf
hatchback
5
210
8.0
5.0
1.5 TSI
1498
150
gasoline
4
Euro 6d
Renault
Clio
hatchback
5
190
11.0
6.5
1.5 dCi
1461
115
diesel
4
Euro 6d-TEMP
Fiat
500
hatchback
3
160
12.9
nan
0.9 TwinAir
875
85
gasoline
2
Euro 6d-TEMP
Peugeot
208
hatchback
nan
190
9.9
5.0
1.2 PureTech
1199
130
gasoline
3
Euro 6d
Volvo
XC60
SUV
5
220
8.0
6.0
B4
nan
nan
nan
nan
nan
Seat
Leon
hatchback
5
240
8.0
nan
1.5 TSI
1498
150
gasoline
4
Euro 6d
Opel
Corsa
hatchback
5
180
9.9
5.5
1.2 Turbo
1197
110
gasoline
4
Euro 6d-TEMP
Ford
Mustang
coupe
2
249
5.8
10.2
2.3L EcoBoost
2261
350
gasoline
4
Euro 6d-TEMP
Volkswagen
Passat
sedan
4
230
7.5
6.0
2.0 TDI
1968
150
diesel
4
Euro 6d-TEMP
Porsche
911
coupe
nan
330
3.5
11.8
3.0L Twin Turbo V6
3000
450
gasoline
6
Euro 6
Audi
Q5
SUV
5
237
6.3
nan
45 TFSI quattro
1984
245
gasoline
4
Euro 6
Volvo
S60
sedan
4
250
6.2
6.6
T8

In [19]:
agr = {}
for tup in param_data['aggregations']:
    name = tup[1] + "_" + tup[0]
    agr[name] = df.agg({tup[0]: tup[1]})[tup[0]]
with open("proj3_ex06_aggregations.json", "w") as file: 
        json.dump(agr, file)

In [20]:
counter = {}
for row in df[param_data['grouping_column']]:
    if row in counter:
        counter[row] += 1
    else:
        counter[row] = 1

print(counter)

grp_column = df[param_data['grouping_column']]
filteredd = df.select_dtypes(include=['number'])
selected_columns = pd.concat([grp_column, filteredd], axis=1)
grouped = selected_columns.groupby(param_data['grouping_column']).mean()

for row in counter:
    if counter[row]<=1:
        grouped = grouped.drop(index=row)

grouped.head(12)
grouped.to_csv('proj3_ex07_groups.csv', header=True, index=True)

{'Audi': 2, 'BMW': 1, 'Volkswagen': 2, 'Renault': 2, 'Fiat': 3, 'Peugeot': 2, 'Volvo': 2, 'Seat': 1, 'Opel': 1, 'Ford': 1, 'Porsche': 1}


In [21]:
pivot_df = pd.pivot_table(df, 
                           index=param_data['pivot_index'], 
                           columns=param_data['pivot_columns'], 
                           values=param_data['pivot_values'], 
                           aggfunc='max')

pivot_df.head(10)
pivot_df.to_pickle('proj3_ex08_pivot.pkl')

In [22]:
long_df = pd.melt(df, id_vars=param_data['id_vars'])
long_df.head(12)
long_df.to_csv('proj3_ex08_melt.csv', header=True, index=False)

In [26]:
dfs = pd.read_csv('proj3_statistics.csv', index_col=0)

dfs.reset_index(inplace=True)

pivot2 = pd.wide_to_long(dfs, stubnames=['Audi', 'BMW', 'Volkswagen', 'Renault'], i='Country', j='Year', sep='_')
    

pivot2.head(20)

pivot2.to_pickle('proj3_ex08_stats.pkl')